In [4]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

In [5]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')
gender_sub = pd.read_csv('gender_submission.csv')

In [6]:
test_data.describe()


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [7]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
gender_sub.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.363636
std,120.810458,0.481622
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [11]:
#kütüphaneleri yükleyelim
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [46]:
#passengerId'yi test setinden sakla (submission için lazım)
test_passenger_ids = test_data["PassengerId"]
full_data = pd.concat([train_data.drop("Survived",axis = 1), test_data], axis = 0)


In [47]:
#Eksil verileri tamamlama
full_data["Age"] = full_data["Age"].fillna(full_data["Age"].median())
full_data["Fare"] = full_data["Fare"].fillna(full_data["Fare"].median())
full_data["Embarked"] = full_data["Embarked"].fillna(full_data["Embarked"].mode(0))

In [48]:
full_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,2.294882,29.503186,0.498854,0.385027,33.281086
std,378.020061,0.837836,12.905241,1.041658,0.865560,51.741500
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,3.000000,35.000000,1.000000,0.000000,31.275000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [49]:
#kategorik verileri sayısala encode etme
full_data["Sex"] = full_data["Sex"].map({"male": 0 , "female":1})
full_data = pd.get_dummies(full_data, columns=["Embarked"], drop_first=True)

In [50]:
full_data = full_data.drop(["PassengerId","Name", "Ticket","Cabin"], axis=1)

In [51]:
#train ve test verisini ayır
x = full_data.iloc[:len(train_data)]
x_test_final = full_data.iloc[len(train_data):]
y = train_data["Survived"]

In [52]:
#Validation ayarı
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [53]:
#random forest
rf_model = RandomForestClassifier(
    n_estimators=200, #
    max_depth=7,
    random_state=42)
rf_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=7, n_estimators=200, random_state=42)

In [54]:
print("Validation Acuuracy", rf_model.score(x_val, y_val))

Validation Acuuracy 0.8156424581005587


In [55]:
predictions = rf_model.predict(x_test_final)

In [57]:
submission= pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": predictions
})

submission.to_csv("submission.cvs",index = False)
print("Submission.csv dosyası oluşturuldu")

Submission.csv dosyası oluşturuldu
